# Analysis of the Gnutella Network

In [19]:
# imports

import os
import random
import warnings
warnings.filterwarnings("ignore")
import powerlaw
import numpy as np
import pandas as pd
from tqdm import tqdm
import networkx as nx
from collections import Counter
from matplotlib import pyplot as plt
import seaborn as sns
import operator


In [20]:
# constructing networks

path = '../data/'
targets = [path + target for target in os.listdir(path)]
S = {idx: nx.read_edgelist(target, delimiter="\t", create_using=nx.DiGraph(name='test')) \
     for idx, target in enumerate(targets)}
for idx, G in enumerate(S):
    S[G].name = targets[idx][-6:-4] + '-08-2002'


In [21]:
# plotting degrees distributions
class Degrees:

    def __init__(self, G):
        self.graph = G
        self.infos = {'graph': self.graph.name,
                      'folder': "-".join(self.graph.name.split("-")[::-1])}
        self._in_degrees = None
        self._out_degrees = None
        self._logbins = None

    def plotter(self):
        # data corpus gathering
        self.constructor()
        self.clustering()

        D = [self._in_degrees, self._out_degrees]
        X = [entry[0] for entry in D]
        Y = [entry[1] for entry in D]
        M = [entry[2] for entry in D]

        #################
        # plotting part #
        #################
        fig, axes = plt.subplots(3, 2, figsize=(17, 17))
        #fig.patch.set_facecolor('w')
        colors = ['bo', 'ro']
        inoutname = ['In', 'Out']
        plt.style.use('dark_background')
        fig.suptitle(f'{self.graph.name} Degree Distribution Plots',  fontsize=36)

        # loglog degree dist. plot
        for idx, ax in enumerate(axes[0]):
            ax.loglog(X[idx], Y[idx], colors[idx])
            ax.set_title(f"LogLog {inoutname[idx]} Degree")
            ax.set_xlabel(M[idx]['xlab'])
            ax.set_ylabel(M[idx]['ylab'])

        # linear degree dist. plot
        for idx, ax in enumerate(axes[1]):
            ax.plot(X[idx], Y[idx], colors[idx])
            ax.set_title(f"Linear {inoutname[idx]} Degree")
            ax.set_xlabel(M[idx]['xlab'])
            ax.set_ylabel(M[idx]['ylab'])

        # log binned degree dist. plot
        #D = self._logbins
        #IN = D[0]; OUT = D[1]; w = D[2]
        #axes[2][0].hist([IN, OUT], bins=np.logspace(np.log10(1), np.log10(max(IN)), num=15), weights=[w,w], log=True, label=["In degree", "Out degree"])
        #axes[2][0].set_yscale("symlog", linthresh=0.0001)
        #axes[2][0].set_xscale("log")
        #axes[2][0].set_title(f"{self.infos['graph']}, Log binning of in degree distribution")
        #axes[2][0].set_xlabel(f"k"); axes[2][0].set_ylabel(f"P(k)"); axes[2][0].legend()

        # ccdf plotting
        D = [self._in_ccdf_data, self._out_ccdf_data]
        bin_centers = [entry[2] for entry in D]
        y = [entry[3] for entry in D]
        for idx, ax in enumerate(axes[2]):
            ax.plot(bin_centers[idx], y[idx], colors[idx])  ## using bin_centers rather than edges
            ax.set_title(f"{inoutname[idx]} Degree CCDF")
            ax.set_xlabel('x')
            ax.set_ylabel('P(x>k)')
            ax.set_yscale("symlog", linthresh=0.00001)
            ax.set_xscale("log")

        return fig

    def constructor(self):
        self.out_degrees()
        self.in_degrees()
        #self.logbins()
        self.ccdf()

    def in_degrees(self):
        freqs = Counter(dict(self.graph.in_degree).values())
        X = freqs.keys()
        Y = freqs.values()
        M = self.infos
        M['title'] = f'degree distribution'
        M['xlab'] = 'k'; M['ylab'] = 'P(k)'
        self._in_degrees = [X, Y, M][:]


    def out_degrees(self):
        freqs = Counter(dict(self.graph.out_degree).values())
        X = freqs.keys()
        Y = freqs.values()
        M = self.infos
        M['title'] = f'degree distribution'
        M['xlab'] = 'k'; M['ylab'] = 'P(k)'
        self._out_degrees = [X, Y, M][:]


    def logbins(self):
        """
        takes in graph.
        saves log binned deg. dist. plots.
        """
        in_deg = list(dict(self.graph.in_degree).values())
        out_deg = list(dict(self.graph.out_degree).values())
        w = np.ones_like(in_deg) / len(out_deg)
        self._logbins = [in_deg, out_deg, w]

    def ccdf(self):
            IN = list(dict(self.graph.in_degree).values())
            n,x, _ = plt.hist(IN, density=True, cumulative=True, bins=100)
            plt.close()
            bin_centers = 0.5*(x[1:]+x[:-1])
            y = [1 - v for v in n]
            self._in_ccdf_data = [n, x, bin_centers, y]

            OUT = list(dict(self.graph.out_degree).values())
            n,x, _ = plt.hist(OUT, density=True, cumulative=True, bins=100)
            plt.close()
            bin_centers = 0.5*(x[1:]+x[:-1])
            y = [1 - v for v in n]
            self._out_ccdf_data = [n, x, bin_centers, y]


    def clustering(self):
        # data setup
        G_cluster = Counter(dict(nx.clustering(self.graph)).values())
        X = G_cluster.keys()
        Y = G_cluster.values()
        Y = [y / max(Y) for y in Y]

        # plotting
        fig, axes = plt.subplots(1, 2, figsize=(14, 7))
        #fig.patch.set_facecolor('w')
        plt.style.use('dark_background')
        fig.suptitle(f'{self.graph.name} Clustering Plots',  fontsize=26)
        axes[0].plot(X, Y, 'o')
        axes[0].set_title('Linear scale')
        axes[0].set_xlabel('C')
        axes[0].set_ylabel('P(C)')
        axes[1].loglog(X, Y, 'o')
        axes[1].set_xlabel('C')
        axes[1].set_ylabel('P(C)')
        axes[1].set_title('LogLog scale')
        plt.savefig(f'../docs/plots/{self.graph.name}/clustering-{self.graph.name}')
        plt.close()

In [22]:
class Centrality:

    def __init__(self, G):
        self.G = G
        self.infos = {'title': self.G.name}
        self._between = None
        self._eigen = None
        self._close = None
        self._indeg = None
        self._outdeg = None
        self.centrality_df = None
        self.L = None
        self.close_dict = None

    def plotter(self):
        self.constructor()
        self.confusion()
        fig, axes = plt.subplots(5, 1, figsize=(14, 35))
        #fig.patch.set_facecolor('w')
        plt.style.use('dark_background')
        fig.suptitle(f'{self.G.name} Centrality Plots',  fontsize=36)


        B = self._between
        axes[0].plot(B[0], B[1], 'co')
        axes[0].set_title('Betweenness Centrality')
        axes[0].set_xlabel(r"$C_d$")
        axes[0].set_ylabel(r"$P(C_d)$")

        E = self._eigen
        axes[1].plot(E[0], E[1], 'co')
        axes[1].set_title('Eigenvector Centrality')
        axes[1].set_xlabel(r"$C_d$")
        axes[1].set_ylabel(r"$P(C_d)$")

        C = self._close
        axes[2].scatter(C[0], C[1])
        axes[2].set_title('Closeness Centrality')
        axes[2].set_xlabel(r"$C_d$")
        axes[2].set_ylabel(r"$P(C_d)$")

        I = self._indeg
        axes[3].plot(I[0], I[1], 'co')
        axes[3].set_title('In Degree Centrality')
        axes[3].set_xlabel(r"$C_d$")
        axes[3].set_ylabel(r"$P(C_d)$")

        O = self._outdeg
        axes[4].plot(O[0], O[1], 'co')
        axes[4].set_title('Out Degree Centrality')
        axes[4].set_xlabel(r"$C_d$")
        axes[4].set_ylabel(r"$P(C_d)$")

        return fig


    def confusion(self):
        v = {}
        v['indeg.'] = []
        v['outdeg.'] = []
        v['betw.'] = []
        v['close'] = []
        v['eigen'] = []
        v['keys'] = []

        for key in self.close_dict.keys():
            v['keys'].append(key)
            v['indeg.'].append(self.indeg_dict[key])
            v['outdeg.'].append(self.outgeg_dict[key])
            v['betw.'].append(self.between_dict[key])
            v['close'].append(self.close_dict[key])
            v['eigen'].append(self.eigen_dict[key])

        centrality_df = pd.DataFrame.from_dict(v, orient='index').transpose()
        centrality_df = centrality_df.set_index('keys')
        self.centrality_df = centrality_df
        corr_mat = centrality_df.astype(float).corr('spearman')
        sns.heatmap(corr_mat, annot = True)
        plt.title('Centrality correlation matrix')
        plt.savefig(f'../docs/plots/{self.G.name}/centrality-confusion-{self.G.name}', transparent=False, dpi=300)
        plt.close()

    def constructor(self):
        self.component()
        self.betweenness()
        self.eigen()
        self.closeness()
        self.indeg()
        self.outdeg()

    def betweenness(self):
        betw = nx.betweenness_centrality(self.G, k=1000)
        in_meta = {"title": f"{self.G.name}, betweenness centrality",
           "folder": "-".join(self.G.name.split("-")[::-1]),
           "file": f"betw-cent-{self.G.name}",
           "xlab": "C_b", "ylab": "P(C_b)"}
        w = np.ones_like(list(betw.values())) / (len(betw.values()))
        n, x, _ = plt.hist(list(betw.values()),  bins = 20, weights = w)
        plt.close()
        bin_centers = 0.5*(x[1:]+x[:-1])
        self.between_dict = betw
        self._between = [bin_centers, n]

    def eigen(self):
        eigen = nx.eigenvector_centrality(self.G, max_iter=200)
        w = np.ones_like(list(eigen.values())) / (len(eigen.values()))
        n, x, _ = plt.hist(list(eigen.values()),  bins = 20, weights = w)
        plt.close()
        bin_centers = 0.5*(x[1:]+x[:-1])
        self.eigen_dict = eigen
        self._eigen = [bin_centers, n]


    def closeness(self):
        clos = {}
        for node in self.sampler():
            clos[node] = nx.closeness_centrality(self.L, u=node)
        in_meta = {"title": f"{self.G.name}, eigenvector centrality",
                   "folder": "-".join(self.G.name.split("-")[::-1]),
                   "file": f"eigen-cent-{self.G.name}",
                   "xlab": "C_b", "ylab": "P(C_b)"}
        w = np.ones_like(list(clos.values())) / (len(clos.values()))
        n, x, _ = plt.hist(list(clos.values()),  bins = 20, weights = w)
        plt.close()
        bin_centers = 0.5*(x[1:]+x[:-1])
        self.close_dict = clos
        self._close = [bin_centers, n]

    def indeg(self):
        indeg = nx.in_degree_centrality(self.G)
        w = np.ones_like(list(indeg.values())) / (len(indeg.values()))
        n, x, _ = plt.hist(list(indeg.values()),  bins = 20, weights = w)
        plt.close()
        bin_centers = 0.5*(x[1:]+x[:-1])
        self.indeg_dict = indeg
        self._indeg = [bin_centers, n]

    def outdeg(self):
        outdeg = nx.out_degree_centrality(self.G)
        w = np.ones_like(list(outdeg.values())) / (len(outdeg.values()))
        n, x, _ = plt.hist(list(outdeg.values()),  bins = 20, weights = w)
        plt.close()
        bin_centers = 0.5*(x[1:]+x[:-1])
        self.outgeg_dict = outdeg
        self._outdeg = [bin_centers, n]

    # helpers
    def component(self):
        S = sorted(nx.strongly_connected_components(self.G), key = len, reverse = True)
        self.L = self.G.subgraph(S[0])
        return self.G.subgraph(S[0])

    def sampler(self):
        N = random.sample(self.L.nodes(), 1000)
        return N

In [23]:
class Fitting:

    def __init__(self, G):
        self.G = G
        self._in_ccdf_data = None
        self._out_ccdf_data = None
        self.in_ccdf = np.array(list(dict(G.in_degree).values()))
        self.out_ccdf = np.array(list(dict(G.out_degree).values()))
        self.fit_in_ccdf = powerlaw.Fit(self.in_ccdf, verbose=False)
        self.fit_out_ccdf = powerlaw.Fit(self.out_ccdf, verbose=False)

    def plotter(self):
        self.constructor()
        fig, axes = plt.subplots(2, 2, figsize=(17, 17))
        #fig.patch.set_facecolor('w')
        inoutname = ['In', 'Out']
        plt.style.use('dark_background')
        fig.suptitle(f'CCDF Fitting For {self.G.name}',  fontsize=36)
        # ccdf wo fit
        D = [self._out_ccdf_data, self._in_ccdf_data]
        bins = [entry[0] for entry in D]
        y = [entry[1] for entry in D]
        for idx, ax in enumerate(axes[0]):
            ax.loglog(bins[idx], y[idx], 'r', linestyle='--', marker='o')
            ax.set_title(f"{inoutname[idx]} CCDF")
            ax.set_xlabel('x')
            ax.set_ylabel('P(x>k)')
            ax.set_yscale("symlog", linthresh=0.00001)
            ax.set_xscale("log")

        # FITTING w. CCDF
        D = [self.fit_in_ccdf, self.fit_out_ccdf]
        for idx, ax in enumerate(axes[1]):
            R, p = D[idx].ccdf()
            ax.plot(R, p, 'r', marker='o')
            D[idx].power_law.plot_ccdf(ax=ax, color='y', label='power-law fit')
            D[idx].exponential.plot_ccdf(ax=ax, color='g', label='exponential fit')
            D[idx].lognormal.plot_ccdf(ax=ax, color='b', label='lognormal fit')
            D[idx].truncated_power_law.plot_ccdf(ax=ax, color='k', label='power-law w. exp. fit')
            ax.legend(loc='upper right')
            ax.set_title(f"Fitting {inoutname[idx]} CCDF")
            ax.set_xlabel('x')
            ax.set_ylabel('P(x>k)')
            ax.set_yscale("symlog", linthresh=0.00001)
            ax.set_xscale("log")
            #ax.ylim(0.001, 1.2)

        return fig

    def constructor(self):
        self.ccdf()

    def ccdf(self):
            IN = list(dict(self.G.in_degree).values())
            n,x, _ = plt.hist(IN, density=True, cumulative=True, bins=100)
            plt.close()
            bin_centers = 0.5*(x[1:]+x[:-1])
            y = [1 - v for v in n]
            self._in_ccdf_data = [bin_centers, y]

            OUT = list(dict(self.G.out_degree).values())
            n,x, _ = plt.hist(OUT, density=True, cumulative=True, bins=100)
            plt.close()
            bin_centers = 0.5*(x[1:]+x[:-1])
            y = [1 - v for v in n]
            self._out_ccdf_data = [bin_centers, y]

In [24]:
class Shorthpath:

    def __init__(self, G):
        self.G = G
        self.L = None
        self.path_lens = None

    def plotter(self):
        self.constructor()
        self.configuration()
        fig, axes = plt.subplots(1, 1, figsize=(10, 10))
        #fig.patch.set_facecolor('w')
        plt.style.use('dark_background')
        axes.bar(list(self.path_lens.keys()), [value / sum(list(self.path_lens.values())) for value in list(self.path_lens.values())])
        fig.suptitle(f'{self.G.name} Shortest Path Length Distributions', fontsize=22)
        axes.set_xlabel('d'); axes.set_ylabel('P(d)')
        return fig


    def constructor(self):
        self.get_largest_components()
        self.get_sample_shorted_paths()

    def get_largest_components(self):
        components_sorted = sorted( nx.strongly_connected_components(self.G) , key=len, reverse=True )
        self.L = self.G.subgraph(components_sorted[0])

    def get_sample_shorted_paths(self):
        path_lens = []

        for i in range(10000):
            nodes = random.sample(self.L.nodes(), 2)
            shortest_path = nx.shortest_path(self.L, source=nodes[0], target=nodes[1])
            path_lens.append(len(shortest_path))
        self.path_lens = Counter(path_lens)
        self.path_lens_2 = path_lens

    def configuration(self):
        C = Configuration(self.G).C
        CL = sorted( nx.strongly_connected_components(C) , key=len, reverse=True)
        CL = C.subgraph(CL[0])
        CL_paths = []

        for i in range(10**4):
            nodes = random.sample(CL.nodes(), 2)
            shortest_path = nx.shortest_path(CL, source=nodes[0], target=nodes[1])
            CL_paths.append(len(shortest_path))

        C_data = CL_paths
        G_data = self.path_lens_2


        fig, ax = plt.subplots(figsize=(8,5))
        ax.hist([G_data, CL_paths], rwidth=0.9, label=['Original', 'Configuration'], color=['red', 'green'])
        ax.legend()
        ax.set_xlabel('d')
        ax.set_ylabel('P(d)')
        plt.title(f"{self.G.name} Shortest Path Configuration")
        plt.savefig(f'../docs/plots/{self.G.name}/config-shorts-{self.G.name}', dpi=300)
        plt.close()


In [25]:
class Configuration:

    def __init__(self, G):
        self.G = G
        self.din = [G.in_degree[node] for node in G.nodes]
        self.dout = [G.out_degree[node] for node in G.nodes]
        self.C = nx.directed_configuration_model(self.din, self.dout)


In [26]:
def run_deg(G):
    # save degree dist plot
    d = Degrees(G)
    fig = d.plotter()
    fig.savefig(f"plots/{G.name}/degrees-{G.name}")
    plt.close()

In [27]:
def run_fit(G):
    # save ccdf fitting plot
    d = Fitting(G)
    fig = d.plotter()
    fig.savefig(f"plots/{G.name}/ccdf-fit-{d.G.name}")
    plt.close()

In [28]:
def run_centrality(G):
    # save centrality dest plot
    b = Centrality(G)
    fig = b.plotter()
    fig.savefig(f"plots/{G.name}/centrality-{b.G.name}")
    plt.close()

In [29]:
def run_short_paths(G):
    # shortest path histogram
    s = Shorthpath(G)
    fig = s.plotter()
    fig.savefig(f"plots/{G.name}/shortest-paths-{s.G.name}")
    plt.close()


In [30]:
def config_cluster(G):
    C = Configuration(G).C
    G_cluster = nx.clustering(G)
    C_cluster = nx.clustering(nx.DiGraph(C))
    G_data = Counter(dict(G_cluster).values())
    C_data = Counter(dict(C_cluster).values())
    G_X = G_data.keys()
    G_Y = G_data.values()
    G_Y = [y / max(G_Y) for y in G_Y]
    C_X = C_data.keys()
    C_Y = C_data.values()
    C_Y = [y / max(C_Y) for y in C_Y]
    fig, axes = plt.subplots(1, 2, figsize=(14, 7))
    #fig.patch.set_facecolor('w')
    plt.style.use('dark_background')
    fig.suptitle(f'{G.name} Clustering Plots (Configuration Model)',  fontsize=26)
    axes[0].plot(G_X, G_Y, 'ro', label='Original')
    axes[0].plot(C_X, C_Y, 'go', label='Configuration')
    axes[0].legend(loc='upper right')
    axes[1].loglog(G_X, G_Y, 'ro', label='Original')
    axes[1].plot(C_X, C_Y, 'go', label='Configuration')
    axes[1].legend(loc='upper right')
    plt.savefig(f'../docs/plots/{G.name}/config-cluster-{G.name}', dpi=300)
    plt.close()

In [36]:
def knn(G):
    C = nx.k_nearest_neighbors(Configuration(G).C)
    D = nx.k_nearest_neighbors(G)
    plt.figure(figsize=(14, 8))
    plt.style.use('dark_background')
    plt.plot(list(D.keys()), list(D.values()), 'ro', label='Original')
    plt.plot(list(C.keys()), list(C.values()), 'bo', label='Configuration')
    plt.legend()
    plt.yscale('log')
    plt.xscale('log')
    plt.xlabel('k')
    plt.ylabel(r'$\bar{k}$')
    plt.title(f'{G.name} KNN Plot', fontsize=26)
    plt.savefig(f'../docs/plots/{G.name}/knn-{G.name}')
    plt.close()


In [32]:
def assortativity(G):
    C = nx.degree_assortativity_coefficient(Configuration(G).C)
    D = nx.degree_assortativity_coefficient(G)
    print(f'original deg. assort: {D:.5f}\nconfmod. def. assort: {C:.5f}')

In [33]:
def comp_size(G):
    return len(sorted(nx.weakly_connected_components(G))[0])

def destruction(G):
    d = np.linspace(0, 0.9, 21)
    yr = []
    yt = []
    x = []
    for i in range(21):
        G = Centrality(G).component()
        N = list(G.nodes)
        R = random.sample(N, int(len(G) * (1 - d[i])))
        S1 = nx.subgraph(G, R)
        T = sorted(dict(G.degree()).items(), key=operator.itemgetter(1))
        T = [d[0] for d in T[::-1][:int(len(G)* (1 - d[i]))]]
        S2 = nx.subgraph(G, T)
        x.append(d[i])
        yr.append(comp_size(S1))
        yt.append(comp_size(S2))
    plt.plot(x, yr, 'b')
    plt.show()
    plt.plot(x, yt, 'r')
    plt.show()

In [34]:
def run_all():
    for G in tqdm(S.values()):
        #run_deg(G)
        #run_fit(G)
        #run_centrality(G)
        #run_short_paths(G)
        #config_cluster(G)
        #assortativity(G)
        knn(G)
        #print(comp_size(G))
        #destruction(G)

In [37]:
run_all()

100%|██████████| 9/9 [00:24<00:00,  2.75s/it]
